<a href="https://colab.research.google.com/github/ishancoderr/3D_modeling/blob/main/add_spatial__reference_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from PIL import Image
import numpy as np


In [15]:
img = Image.open("/content/categories.tif")
img_array = np.array(img)


In [16]:
single_band_array = np.zeros((img_array.shape[0], img_array.shape[1]), dtype=np.uint8)


In [17]:
for i in range(img_array.shape[0]):
    for j in range(img_array.shape[1]):
        if np.all(img_array[i,j,:] == [0, 0, 255]):  # blue
            single_band_array[i,j] = 1
        elif np.all(img_array[i,j,:] == [255, 255, 255]):  # white
            single_band_array[i,j] = 2
        elif np.all(img_array[i,j,:] == [255, 255, 0]):  # yellow
            single_band_array[i,j] = 3
        elif np.all(img_array[i,j,:] == [255, 0, 0]):  # red
            single_band_array[i,j] = 4


In [19]:
single_band_img = Image.fromarray(single_band_array)
single_band_img.save("/content/output_image.tif")


In [23]:
from osgeo import gdal, osr


src_ds = gdal.Open('/content/Slope.tif')
dst_ds = gdal.Open('/content/output_image.tif', gdal.GA_Update)

# Get the geotransform and spatial reference information from the source file
src_geotransform = src_ds.GetGeoTransform()
src_proj = src_ds.GetProjection()
src_srs = osr.SpatialReference()
src_srs.ImportFromWkt(src_proj)


dst_ds.SetGeoTransform(src_geotransform)
dst_ds.SetProjection(src_proj)

src_ds = None
dst_ds = None

In [24]:

ds = gdal.Open('/content/output_image.tif')

geotransform = ds.GetGeoTransform()
projection = ds.GetProjection()

print('Geotransform:', geotransform)
print('Projection:', projection)

ds = None

Geotransform: (428122.4950000001, 14.030000000000005, 0.0, 755741.5723999995, 0.0, -14.030000000000017)
Projection: PROJCS["WGS 84 / UTM zone 44N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32644"]]


In [26]:
!pip install geopandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.8 MB/s eta 0:00:00


In [27]:
import geopandas as gpd

# Read the shapefile into a GeoDataFrame
gdf = gpd.read_file('/content/RiskMap_ShapeFile.shp')

# Drop unnecessary columns to make the GeoDataFrame lighter
gdf = gdf[['geometry']]

# Convert the GeoDataFrame to GeoJSON and write it to a file
gdf.to_file('/content/output.geojson', driver='GeoJSON')


In [34]:
from osgeo import gdal

# Open the tiff file
ds = gdal.Open('/content/Reclassified.tif')

# Get the geotransform and spatial reference information from the file
geotransform = ds.GetGeoTransform()
srs = osr.SpatialReference()
srs.ImportFromWkt(ds.GetProjection())

# Define the point to check
lon = 6.68278 
lat = 80.39917

# Convert the point's coordinates to the same projection as the TIFF file
point_srs = osr.SpatialReference()
point_srs.ImportFromEPSG(4326)  # WGS 84
transform = osr.CoordinateTransformation(point_srs, srs)
point = transform.TransformPoint(lon, lat)

# Check if the point is within the extent of the TIFF file
x, y = point[0], point[1]
if (x >= geotransform[0] and x <= geotransform[0] + geotransform[1] * ds.RasterXSize and 
    y <= geotransform[3] and y >= geotransform[3] + geotransform[5] * ds.RasterYSize):
    print("The point is within the extent of the TIFF file.")
else:
    print("The point is not within the extent of the TIFF file.")
    
# Close the file
ds = None


The point is within the extent of the TIFF file.


In [38]:
from osgeo import gdal

# Open the tiff file
ds = gdal.Open('/content/Reclassified.tif')

# Get the geotransform and spatial reference information from the file
geotransform = ds.GetGeoTransform()
srs = osr.SpatialReference()
srs.ImportFromWkt(ds.GetProjection())

# Define the point to check
lon = 6.747836
lat = 80.664644

# Convert the point's coordinates to the same projection as the TIFF file
point_srs = osr.SpatialReference()
point_srs.ImportFromEPSG(4326)  # WGS 84
transform = osr.CoordinateTransformation(point_srs, srs)
point = transform.TransformPoint(lon, lat)

# Check if the point is within the extent of the TIFF file
x, y = point[0], point[1]
if (x >= geotransform[0] and x <= geotransform[0] + geotransform[1] * ds.RasterXSize and 
    y <= geotransform[3] and y >= geotransform[3] + geotransform[5] * ds.RasterYSize):
    print("The point is within the extent of the TIFF file.")
    
    # Get the raster band
    band = ds.GetRasterBand(1)
    
    # Get the value of the pixel at the location
    px = int((x - geotransform[0]) / geotransform[1])
    py = int((y - geotransform[3]) / geotransform[5])
    pixel_value = band.ReadAsArray(px, py, 1, 1)[0][0]
    
    print("Pixel value at the location is:", pixel_value)
    
else:
    print("The point is not within the extent of the TIFF file.")
    
# Close the file
ds = None
print("GDAL version:", gdal.__version__)

The point is within the extent of the TIFF file.
Pixel value at the location is: 3
GDAL version: 3.3.2


In [40]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.7 MB/s eta 0:00:00


In [42]:
import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)


Python version
3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
Version info.
sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)
